# 03 - Simulation des municipales Paris 2026

Interface principale de simulation avec dashboard interactif.

## Réforme d'août 2025

- **Scrutin 1** : Liste unique parisienne → Conseil de Paris (163 sièges, prime 25%)
- **Scrutin 2** : 17 scrutins sectoriels → conseils d'arrondissement (prime 50%)

## Contenu

1. Dashboard interactif (sliders)
2. Simulation manuelle avec scénarios
3. Visualisation hémicycle
4. Cartes par secteur

In [ ]:
import sys
sys.path.insert(0, '..')

import warnings
warnings.filterwarnings('ignore', category=UserWarning)  # Sièges provisionnels

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.display import display, HTML

from paris_elections.engine.simulation import ElectionSimulator
from paris_elections.scenarios.scenario import (
    Scenario,
    scenario_gauche_unie,
    scenario_droite_unie,
    scenario_fragmentation,
)
from paris_elections.viz.hemicycle import plot_hemicycle
from paris_elections.viz.maps import map_winners
from paris_elections.config import CONSEIL_PARIS_SEATS, MAYOR_ABSOLUTE_MAJORITY

## 1. Dashboard interactif

Ajustez les sliders et cliquez sur **Simuler** pour voir les résultats en temps réel.

In [ ]:
try:
    from paris_elections.viz.dashboard import create_dashboard
    dashboard = create_dashboard()
    dashboard.display()
except ImportError as e:
    print(f"Dashboard non disponible : {e}")
    print("→ Utiliser les cellules suivantes pour la simulation manuelle.")

## 2. Simulation manuelle

In [ ]:
# Définir les scores manuellement
scores_paris = {
    "PS": 20.0,
    "LFI": 12.0,
    "EELV": 7.0,
    "PCF": 3.0,
    "REN": 18.0,
    "LR": 17.0,
    "RN": 9.0,
    "REC": 5.0,
    "DIV": 9.0,
}

participation = 0.45

print(f"Total des scores : {sum(scores_paris.values()):.1f}%")
print(f"Participation : {participation*100:.0f}%")

In [ ]:
# Exécuter la simulation
sim = ElectionSimulator()
result = sim.run(
    paris_scores=scores_paris,
    sector_scores={},  # Mêmes scores pour tous les secteurs
    participation=participation,
)

# Résultats Conseil de Paris
seats = result.total_seats_conseil
print("\n" + "="*50)
print(f"CONSEIL DE PARIS — {sum(seats.values())} sièges")
print("="*50 + "\n")

for liste in sorted(seats.keys(), key=lambda x: seats[x], reverse=True):
    n = seats[liste]
    pct = n / CONSEIL_PARIS_SEATS * 100
    bar = "█" * (n // 3)
    print(f"{liste:6} : {n:3} sièges ({pct:5.1f}%) {bar}")

print("\n" + "-"*50)
max_liste = max(seats, key=seats.get)
max_seats = seats[max_liste]
print(f"🏆 Liste en tête : {max_liste} ({max_seats} sièges)")
if max_seats >= MAYOR_ABSOLUTE_MAJORITY:
    print(f"✅ Majorité absolue ({MAYOR_ABSOLUTE_MAJORITY} requis)")
else:
    print(f"❌ Pas de majorité ({MAYOR_ABSOLUTE_MAJORITY - max_seats} sièges manquants)")

## 3. Visualisation hémicycle

In [ ]:
fig = plot_hemicycle(seats, title=f"Conseil de Paris — {CONSEIL_PARIS_SEATS} sièges")
plt.show()

## 4. Résultats par secteur

In [ ]:
# Tableau des vainqueurs par secteur
sectors_summary = []
for secteur, arr_result in result.arrondissements.items():
    sectors_summary.append({
        'Secteur': secteur,
        'Vainqueur': arr_result.winner,
        'Sièges total': arr_result.total_seats,
        'Sièges vainqueur': arr_result.seats.get(arr_result.winner, 0) if arr_result.winner else 0,
    })

sectors_df = pd.DataFrame(sectors_summary)
display(sectors_df)

In [ ]:
# Carte des vainqueurs
winners = {row['Secteur']: row['Vainqueur'] for _, row in sectors_df.iterrows()}
m = map_winners(winners)
m

## 5. Scénarios prédéfinis

In [ ]:
# Charger les scénarios prédéfinis
scenarios = [
    scenario_gauche_unie(),
    scenario_droite_unie(),
    scenario_fragmentation(),
]

for sc in scenarios:
    print(f"\n{'='*50}")
    print(f"📊 Scénario : {sc.name}")
    print(f"   {sc.description}")
    print(f"{'='*50}")
    
    res = sc.simulate()
    seats_sc = res.total_seats_conseil
    
    for liste in sorted(seats_sc.keys(), key=lambda x: seats_sc[x], reverse=True)[:5]:
        print(f"   {liste}: {seats_sc[liste]} sièges")

## 6. Export des résultats

In [ ]:
# Export JSON du scénario
scenario_custom = Scenario(
    name="Scénario personnalisé",
    description="Simulation manuelle depuis le notebook",
    paris_scores=scores_paris,
    participation=participation,
)

# Afficher le JSON
print(scenario_custom.to_json())

---

## Notes

⚠️ **Les effectifs des conseils d'arrondissement sont provisionnels** (basés sur les données 2020). À actualiser lors de la publication du décret d'application de la réforme de 2025.

➡️ **Notebook 04** : Comparaison multi-scénarios et Monte Carlo